In [1]:
#from csv_psfs import *
import svd_model as svm

import forward_model as fm

import numpy as np
import scipy.io

# for a bit of investigation into the process surrounding the SVD
#import pdb
from IPython.core.debugger import set_trace

# load my own module for loading the PSFs
import sys
sys.path.append("../tensorflow")
import load_PSFs

# My SVD
Grab the PSF stack, do the SVD, test on the hydra. Simple as.

In [2]:
psf_directory = "/home/dshteinbok/nV3_PSFs"
psf_meta = "/home/dshteinbok/nV3_PSFs_meta/PSF_Shifts.csv"
# TODO: where to save

# (height, width)
#img_dims = (800, 1280)

# ACTUALLY should be (width, height) to be compatible with load_PSFs
img_dims = (1280, 800)

In [3]:
# load the PSFs
#psfs = pad_as_center(load_from_dir(psf_directory), img_dims[0], img_dims[1])
psfs = load_PSFs.load_PSFs_csv(psf_directory, psf_meta, img_dims)[:,:,:,0]

caught error
caught error
caught error
caught error
caught error
caught error
caught error
caught error
caught error
caught error
caught error
caught error
caught error
caught error


**ERROR DISCOVERED IN WALLER PAPER:** Waller et al. basically use the location of the brightest point in each of the PSFs to interpolate the PSF for all other points (specifically they interpolate the weights by which they must multiply each "kernel" for all points from the ones that they observe). However, they actually have all their PSFs centered in the frame, so all but one of their PSFs are in the center. The other PSF is (1,1) away from the center. Thus, all points within any quadrant exhibit shift-invariance.

To solve this, you need to be careful about how you pad your PSF. You don't want it to be in the center, but rather you want it shifted to wherever the light ends up.

In our case, according to the spec, this should be 1 pixel for every 0.8 um. 

In [4]:
ref_image_index=4
set_trace()
psfs_reg,shifts=svm.register_psfs(psfs, psfs[:,:,ref_image_index],dct_on=True)

--Return--
None
> /tmp/ipykernel_3432064/573455527.py(2)<cell line: 2>()
      1 ref_image_index=4
----> 2 set_trace()
      3 psfs_reg,shifts=svm.register_psfs(psfs, psfs[:,:,ref_image_index],dct_on=True)

ipdb> c
Removing background

done

registering

done registering



In [5]:
rank=12
comps, weights_interp=svm.calc_svd(psfs_reg,shifts,rank)

creating matrix

done

starting svd...

check values of ymat
interpolating...

done




In [6]:
# Normalize weights to have maximum sum through rank of 1
weights_norm = np.sum(weights_interp[weights_interp.shape[0]//2-1,weights_interp.shape[1]//2-1,:],0).max()
weights = weights_interp/weights_norm;

#normalize by norm of all stack. Can also try normalizing by max of all stack or by norm of each slice
h=comps/np.linalg.norm(np.ravel(comps))

In [7]:
h.shape

(800, 1280, 12)

In [8]:
weights.shape

(800, 1280, 12)

In [9]:
h_dict = {"array_out": h}
scipy.io.savemat("../data/nV3_h.mat", h_dict)

In [10]:
weights_dict = {"array_out": weights}
scipy.io.savemat("../data/nV3_weights.mat", weights_dict)

Now, view each of the PSFs at the 12 positions: